# Neighborhoods in Toronto

### Installing/Importing required libraries

In [1]:
pip install geocoder

     |████████████████████████████████| 102kB 2.7MB/s ta 0:00:011
     |████████████████████████████████| 92kB 5.1MB/s eta 0:00:011
     |████████████████████████████████| 829kB 3.0MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment acthis line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm  # Matplotlib and associated plotting modules
import matplotlib.colors as colors

from sklearn.cluster import KMeans   # import k-means from clustering stage

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    brotlipy-0.7.0             |py36h8c4c3a4_1000  

### Setting display options

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Installing BeautifulSoup4 for web scraping

In [4]:
#pip3 install beautifulsoup4
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.0       |   py36h9f0ad1d_0         160 KB  conda-forge
    soupsieve-1.9.4            |   py36h9f0ad1d_1          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         218 KB

The following NEW packages will be INSTALLED:

  beautifulsoup4     conda-forge/linux-64::beautifulsoup4-4.9.0-py36h9f0ad1d_0
  soupsieve          conda-forge/linux-64::soupsieve-1.9.4-py36h9f0ad1d_1



soupsieve-1.9.4      | 58 KB     | ##################################### | 100% 
beautifulsoup4-4.9.0 | 160 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction

### Part 1 Web Scraping

In [5]:
from bs4 import BeautifulSoup

URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(URL).text

soup = BeautifulSoup(page, 'html.parser')

In [6]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XrRJvgpAEKcAAI@-EI0AAAAQ","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":955414546,"wgRevisionId":955414546,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario

### Part 2 Getting table heading

In [ ]:
gdp_table = soup.find("table", attrs={"class": "wikitable"})
gdp_table_data = gdp_table.tbody.find_all("tr")
headings = []
for td in gdp_table_data[0].find_all("th"):
    # remove any newlines and extra spaces from left and right
    headings.append(td.text.replace('\n', ' ').strip())

In [8]:
print(headings)

['Postal Code', 'Borough', 'Neighborhood']


### Part 3 Extracting data and moving to DataFrame

In [9]:
t_headers = []
for x in range(0,len(gdp_table_data)):
    for td in gdp_table_data[x].find_all("td"):
        # remove any newlines and extra spaces from left and right
        t_headers.append(td.text.replace('\n', ' ').strip())

count=0
dict={headings[0]:[],headings[1]:[],headings[2]:[]}
for x in range(0,int(len(t_headers)/3)):
    dict[headings[0]].append(t_headers[count])
    dict[headings[1]].append(t_headers[count+1])
    dict[headings[2]].append(t_headers[count+2])
    count=count+3
df=pd.DataFrame(dict)
df=df[df['Borough']!='Not assigned']
df=df.sort_values('Postal Code')
df.reset_index(drop=True,inplace=True)

In [10]:
#import geocoder # import geocoder
# initialize your variable to None
#lat_lng_coords = None
#postal_code='M5A'
# loop until you get the coordinates
#while(lat_lng_coords is None):
#g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#lat_lng_coords = g.latlng
#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

### Part 4 Reading coordinates for Data

In [11]:
dfs=pd.read_csv('Geospatial_Coordinates.csv')
dfs.sort_values('Postal Code').reset_index(drop=True,inplace=True)
dfs=dfs.set_index('Postal Code')

In [12]:
lat=[]
lng=[]
for x in df['Postal Code']:
    lat.append(dfs.loc[x].Latitude)
    lng.append(dfs.loc[x].Longitude)
df['Latitude']=lat
df['Longitude']=lng

In [13]:
dfs.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [14]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
df.shape

(103, 5)

### Part 5 Selecting data only for Toronto 

In [16]:
tdf=df[df['Borough'].str.contains('Toronto')]
tdf.reset_index(drop=True,inplace=True)

In [17]:
tdf.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Part 6 Create map to show different Borough

In [18]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.6534817, -79.3839347.


In [20]:
list1=list(tdf.Borough.unique())
colorlist=['red','blue','yellow','green']

# create map of Manhattan using latitude and longitude values
toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
count=0
for x in list1:
    temp=tdf[tdf['Borough']==x]
    # add markers to map
    for lat, lng, label in zip(temp['Latitude'], temp['Longitude'], temp['Neighborhood']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=colorlist[count],
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(toronto)  
    count+=1
toronto

### Part 7 Get all the venue near these Borough's Neighborhood

In [21]:
CLIENT_ID = 'TAM0AZCRXZJVGTBEB5TREJNKTEMYWKXHQX5NWIG4WS0O1LOW' # your Foursquare ID
CLIENT_SECRET = 'K4UTSYZSKXJKC5VIQVU1NNMQI5JFN451LH0I23IFYJOZE44E' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TAM0AZCRXZJVGTBEB5TREJNKTEMYWKXHQX5NWIG4WS0O1LOW
CLIENT_SECRET:K4UTSYZSKXJKC5VIQVU1NNMQI5JFN451LH0I23IFYJOZE44E


In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
# type your answer here

toronto_venues = getNearbyVenues(names=tdf['Neighborhood'],
                                   latitudes=tdf['Latitude'],
                                   longitudes=tdf['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South
Parkdale, Ron

In [39]:
toronto_venues[toronto_venues['Venue Category']=='Neighborhood']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
65,Studio District,43.659526,-79.340923,Leslieville,43.662070,-79.337856,Neighborhood
393,"Richmond, Adelaide, King",43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood
414,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,Harbourfront,43.639526,-79.380688,Neighborhood


In [26]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
Business reply mail Processing Centre,17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,30,30,30,30,30,30
Christie,17,17,17,17,17,17
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,30,30,30,30,30,30


In [27]:
len(toronto_venues['Venue Category'].unique())

190

In [33]:
toronto_venues['Venue Category'].unique()

array(['Trail', 'Health Food Store', 'Pub', 'Park', 'Neighborhood',
       'Cosmetics Shop', 'Greek Restaurant', 'Italian Restaurant',
       'Ice Cream Shop', 'Yoga Studio', 'Brewery',
       'Fruit & Vegetable Store', 'Dessert Shop', 'Restaurant',
       'Pizza Place', 'Juice Bar', 'Bookstore', 'Bubble Tea Shop',
       'Furniture / Home Store', 'Grocery Store', 'Spa', 'Coffee Shop',
       'Bakery', 'Caribbean Restaurant', 'Gym', 'Fish & Chips Shop',
       'Fast Food Restaurant', 'Sushi Restaurant', 'Liquor Store',
       'Pet Store', 'Steakhouse', 'Burrito Place', 'Movie Theater',
       'Sandwich Place', 'Intersection', 'Food & Drink Shop',
       'Fish Market', 'Gay Bar', 'Seafood Restaurant',
       'American Restaurant', 'Cheese Shop', 'Café',
       'Middle Eastern Restaurant', 'Comfort Food Restaurant',
       'Stationery Store', 'Wine Bar', 'Thai Restaurant',
       'Coworking Space', 'Latin American Restaurant', 'Swim School',
       'Bus Line', 'Breakfast Spot', 'Departme

### Part 8 Create all the categories as columns

In [93]:
catdf=pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
catdf.rename(columns={'Neighborhood':'Neighborhoodd'},inplace=True)
cols=['Neighborhood']+list(catdf.columns)
catdf['Neighborhood']=toronto_venues['Neighborhood']
catdf=catdf[cols]
catdf=catdf.groupby('Neighborhood').mean().reset_index()

In [94]:
catdf.head()

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhoodd,New American Restaurant,Nightclub,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.033333,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.033333,0.0,0.000000,0.033333,0.066667,0.0,0.0,0.033333,0.0000,0.0,0.0000,0.033333,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.033333,0.0,0.0,0.0,0.033333,0.0,0.0,0.000000,0.0,0.066667,0.066667,0.0,0.0,0.0,0.0,0.033333,0.000000,0.033333,0.000000,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.000000,0.0,0.033333,0.0,0.0,0.0,0.0,0.033333,0.033333,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.033333,0.0,0.0,0.0,0.0,0.0,0.000000,0.033333,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.033333,0.0,0.0,0.0,0.000000,0.0,0.0,0.033333,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0000,0.033333,0.0,0.0,0.0,0.000000,0.0000,0.066667,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.033333,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.045455,0.0,0.045455,0.000000,0.0

In [95]:
catdf.shape

(39, 191)

### Part 9 Get most common venue category

In [96]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [97]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = catdf['Neighborhood']

for ind in np.arange(catdf.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(catdf.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cocktail Bar,Beer Bar,Coffee Shop,Seafood Restaurant,Breakfast Spot,Basketball Stadium,Farmers Market,Bistro,Concert Hall,Jazz Club
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Nightclub,Restaurant,Italian Restaurant,Stadium,Intersection,Bar,Bakery
2,Business reply mail Processing Centre,Pizza Place,Auto Workshop,Comic Shop,Recording Studio,Restaurant,Burrito Place,Light Rail Station,Brewery,Skate Park,Spa
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Airport,Bar,Harbor / Marina,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique
4,Central Bay Street,Coffee Shop,Café,Discount Store,Bar,Ice Cream Shop,Spa,Japanese Restaurant,Sandwich Place,Bubble Tea Shop,Yoga Studio


### Part 10 Use K-Means to cluster all the categories into 5 different clusters

In [98]:
cluster=catdf.drop('Neighborhood',1)
kmeans = KMeans(n_clusters=5, random_state=0).fit(cluster)
kmeans.labels_

In [ ]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
tdf = tdf.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [109]:
tdf.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Park,Pub,Trail,Health Food Store,Neighborhoodd,Yoga Studio,Dance Studio,Donut Shop,Dog Run,Distribution Center
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,3,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Fruit & Vegetable Store,Pub,Pizza Place,Juice Bar,Grocery Store,Furniture / Home Store,Dessert Shop
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,3,Sandwich Place,Fast Food Restaurant,Pet Store,Brewery,Burrito Place,Restaurant,Liquor Store,Italian Restaurant,Intersection,Ice Cream Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Bakery,Ice Cream Shop,Gay Bar,Bookstore,Seafood Restaurant,Sandwich Place,Brewery,Fish Market
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Swim School,Bus Line,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center


In [110]:
tdf.shape

(39, 16)

### Part 11 Map to show all the cluster in Neighborhood of Toronto Borough

In [123]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
colors_array = ['yellow','red','blue','black','green']

# add markers to the map
for lat, lon, poi, cluster in zip(tdf['Latitude'], tdf['Longitude'], tdf['Neighborhood'], tdf['Cluster Labels']):
    label = folium.Popup(str(poi) + ', Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colors_array[cluster-1],
        fill=True,
        fill_color=colors_array[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 0 - Cafe/Coffee Shops

In [129]:
tdf.loc[tdf['Cluster Labels'] == 0, tdf.columns[[1] + list(range(5, tdf.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,0,Café,Coffee Shop,Bakery,Ice Cream Shop,Gay Bar,Bookstore,Seafood Restaurant,Sandwich Place,Brewery,Fish Market
13,Downtown Toronto,0,Coffee Shop,Park,Breakfast Spot,Theater,Bakery,Ice Cream Shop,Farmers Market,Event Space,Distribution Center,Spa
14,Downtown Toronto,0,Café,Theater,Coffee Shop,Shopping Mall,Sporting Goods Shop,Electronics Store,College Rec Center,Sandwich Place,Diner,Restaurant
15,Downtown Toronto,0,Gastropub,Café,Coffee Shop,Cocktail Bar,BBQ Joint,Cosmetics Shop,Creperie,Middle Eastern Restaurant,Diner,Restaurant
17,Downtown Toronto,0,Coffee Shop,Café,Discount Store,Bar,Ice Cream Shop,Spa,Japanese Restaurant,Sandwich Place,Bubble Tea Shop,Yoga Studio
18,Downtown Toronto,0,Café,Coffee Shop,Seafood Restaurant,Neighborhoodd,Pizza Place,Japanese Restaurant,Deli / Bodega,Colombian Restaurant,Smoke Shop,Speakeasy
20,Downtown Toronto,0,Coffee Shop,Café,Japanese Restaurant,Concert Hall,Pizza Place,Sandwich Place,Restaurant,Bookstore,Pub,Beer Bar
21,Downtown Toronto,0,Café,Deli / Bodega,Restaurant,Japanese Restaurant,Coffee Shop,Gastropub,Bakery,Sandwich Place,Seafood Restaurant,Pub
24,Central Toronto,0,Sandwich Place,Café,Coffee Shop,Pharmacy,Cosmetics Shop,Donut Shop,Liquor Store,Indian Restaurant,Pub,Middle Eastern Restaurant
25,Downtown Toronto,0,Café,Japanese Restaurant,Restaurant,Bookstore,Italian Restaurant,Bar,Bakery,Sushi Restaurant,Comfort Food Restaurant,College Gym


## Cluster 1 - Parks

In [130]:
tdf.loc[tdf['Cluster Labels'] == 1, tdf.columns[[1] + list(range(5, tdf.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,1,Restaurant,Playground,Park,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
10,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Dance Studio,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store
23,Central Toronto,1,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Donut Shop,Dog Run


## Cluster 2 - Miscellaneous

In [131]:
tdf.loc[tdf['Cluster Labels'] == 2, tdf.columns[[1] + list(range(5, tdf.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,2,Park,Swim School,Bus Line,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center


## Cluster 3 - Restaurants

In [132]:
tdf.loc[tdf['Cluster Labels'] == 3, tdf.columns[[1] + list(range(5, tdf.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,3,Park,Pub,Trail,Health Food Store,Neighborhoodd,Yoga Studio,Dance Studio,Donut Shop,Dog Run,Distribution Center
1,East Toronto,3,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Fruit & Vegetable Store,Pub,Pizza Place,Juice Bar,Grocery Store,Furniture / Home Store,Dessert Shop
2,East Toronto,3,Sandwich Place,Fast Food Restaurant,Pet Store,Brewery,Burrito Place,Restaurant,Liquor Store,Italian Restaurant,Intersection,Ice Cream Shop
5,Central Toronto,3,Breakfast Spot,Park,Gym,Sandwich Place,Food & Drink Shop,Hotel,Department Store,Yoga Studio,Dessert Shop,Eastern European Restaurant
6,Central Toronto,3,Clothing Store,Coffee Shop,Sporting Goods Shop,Seafood Restaurant,Salon / Barbershop,Restaurant,Café,Chinese Restaurant,Pet Store,Park
7,Central Toronto,3,Dessert Shop,Pizza Place,Sandwich Place,Gym,Italian Restaurant,Café,Sushi Restaurant,Coffee Shop,Brewery,Restaurant
9,Central Toronto,3,Pub,Coffee Shop,Fried Chicken Joint,Light Rail Station,Liquor Store,Restaurant,Sports Bar,Bank,Supermarket,Sushi Restaurant
11,Downtown Toronto,3,Bakery,Restaurant,Café,Italian Restaurant,Coffee Shop,Pet Store,Deli / Bodega,Caribbean Restaurant,Pub,Playground
12,Downtown Toronto,3,Bookstore,Indian Restaurant,Hobby Shop,Juice Bar,Beer Bar,Café,Pub,Smoke Shop,Ramen Restaurant,Sushi Restaurant
16,Downtown Toronto,3,Cocktail Bar,Beer Bar,Coffee Shop,Seafood Restaurant,Breakfast Spot,Basketball Stadium,Farmers Market,Bistro,Concert Hall,Jazz Club


## Cluster 4 - Health/Beauty

In [133]:
tdf.loc[tdf['Cluster Labels'] == 4, tdf.columns[[1] + list(range(5, tdf.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,4,Health & Beauty Service,Garden,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store
